In [8]:
import pandas as pd
import numpy as np
import scipy.io
import math
from collections import Counter
import copy
import warnings
import tensorflow 
from tensorflow.keras.layers import Dense, Input,Conv1D, Flatten, Dropout
from sklearn.preprocessing import StandardScaler
from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import BatchNormalization
warnings.filterwarnings("ignore")
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt

In [39]:
def consecutive(data, stepsize=1):
    return np.split(data, np.where(np.diff(data) != stepsize)[0] + 1)
def pretty(d, indent=0):
    for key, value in d.items():
        print('\t' * indent + str(key))
        if isinstance(value, dict):

            pretty(value, indent + 2)
            print('-----------------------')
        else:
            print('\t' * (indent + 2) + str(value))
            print(" ")
def most_frequent(List):
    occurence_count = Counter(List)
    return occurence_count.most_common(1)[0][0]
def rms(arr):
    n = len(arr)
    squared = np.array(arr) * np.array(arr)
    sum = np.sum(squared)
    mean = (sum / (float)(n))
    root = math.sqrt(mean)
    return root

def extractSubject(name):
    ex1Path = 'DB1/' + name + '/' + name + '_A1_E1.mat'
    print(ex1Path)
    ex1 = scipy.io.loadmat(ex1Path)
    emg = ex1['emg']
    EMGdf = pd.DataFrame.from_dict(emg)
    stimulus = ex1['stimulus']

    ex2Path = 'DB1/' + name + '/' + name + '_A1_E2.mat'
    ex2 = scipy.io.loadmat(ex2Path)
    emg2 = ex2['emg']
    EMGdf2 = pd.DataFrame.from_dict(emg2)
    stimulus2 = ex2['stimulus']

    ex3Path = 'DB1/' + name + '/' + name + '_A1_E3.mat'
    ex3 = scipy.io.loadmat(ex3Path)
    emg3 = ex3['emg']
    EMGdf3 = pd.DataFrame.from_dict(emg3)
    stimulus3 = ex3['stimulus']

    Movements = {}
    for m in range(1, 53):
        if (m < 13):
            movementIndices = np.where(stimulus == m)[0]
            repetitions = consecutive(movementIndices)
            EMG = EMGdf
        elif (m < 30):
            movementIndices = np.where(stimulus2 == (m - 12))[0]
            repetitions = consecutive(movementIndices)
            EMG = EMGdf2
        else:
            movementIndices = np.where(stimulus3 == (m - 29))[0]
            repetitions = consecutive(movementIndices)
            EMG = EMGdf3

        Repetitions = {}
        for r in range(1, 7):
            startIndex = repetitions[r - 1][0]
            LastIndex = repetitions[r - 1][len(repetitions[r - 1]) - 1]
            df = EMG.iloc[startIndex:LastIndex, 0:10]
            df.reset_index(drop=True, inplace=True)
            Repetitions["R{0}".format(r)] = df
        Movements["Movement{0}".format(m)] = Repetitions
    return Movements
def Average(lst):
    return sum(lst) / len(lst)




In [61]:
columns = {'E1', 'E2', 'E3', 'E4', 'E5', 'E6', 'E7', 'E8', 'E9', 'E10', 'Train','Movement'}
training_set = []
y_train = []
testing_set = []
y_test = []
for s in range(1,2):
    subject = 'S' + str(s)
    dff = pd.DataFrame.from_dict(extractSubject(subject))
    for m in range(1,53):
        M = dff['Movement'+str(m)]
        for r in range(1,7): 
            r_index = "R" + str(r)
            rep = M[r_index]
            if (r in [1, 3, 4, 6]):
                train = True
            else:
                train = False
            for x in range(0, len(rep), 6):
                entry = rep.iloc[x:x+15, :]
                entry.columns = {'E1', 'E2', 'E3', 'E4', 'E5', 'E6', 'E7', 'E8', 'E9', 'E10'}
                entry.reset_index(drop=True, inplace=True)
                print(entry)
                if(train):
                   training_set.append(entry)
                   y_train.append(m)
                else :
                   testing_set.append(entry)
                   y_test.append(m)  






    


DB1/S1/S1_A1_E1.mat
        E7     E10      E4      E1      E6      E9      E2      E3      E5  \
0   0.0146  0.0024  0.0049  0.0024  0.0024  0.0024  0.0024  0.0171  0.0024   
1   0.0171  0.0024  0.0024  0.0024  0.0024  0.0024  0.0024  0.0220  0.0024   
2   0.0342  0.0024  0.0024  0.0024  0.0024  0.0024  0.0024  0.0269  0.0024   
3   0.0439  0.0024  0.0024  0.0024  0.0024  0.0024  0.0024  0.0293  0.0024   
4   0.0464  0.0024  0.0024  0.0024  0.0024  0.0024  0.0024  0.0293  0.0024   
5   0.0464  0.0024  0.0024  0.0024  0.0024  0.0024  0.0024  0.0269  0.0024   
6   0.0439  0.0024  0.0024  0.0024  0.0024  0.0024  0.0024  0.0244  0.0024   
7   0.0391  0.0024  0.0024  0.0024  0.0024  0.0024  0.0024  0.0244  0.0024   
8   0.0342  0.0024  0.0024  0.0024  0.0024  0.0024  0.0024  0.0220  0.0024   
9   0.0269  0.0024  0.0024  0.0024  0.0024  0.0024  0.0024  0.0220  0.0024   
10  0.0195  0.0024  0.0024  0.0024  0.0024  0.0024  0.0024  0.0195  0.0024   
11  0.0146  0.0024  0.0024  0.0024  0.0024  

In [62]:
print(y_train)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [17]:
X_train = np.asarray(X_train).astype('float64')
y_train = np.asarray(y_train).astype('float64')
X_test = np.asarray(X_test).astype('float64')
y_test = np.asarray(y_test).astype('float64')

X_train = np.array(X_train).reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = np.array(X_test).reshape(X_test.shape[0], X_test.shape[1], 1)
print("X Train shape: ", X_train.shape)
print("X Test shape: ", X_test.shape)
input = Input(shape =(10,1))
x = Conv1D(32,10, activation='relu', padding='same', input_shape = (X_train.shape[1],1))(input)
x = Conv1D(64,31, activation='relu' ,padding='same' )(x)
x = BatchNormalization()(x)
x = Conv1D(64,31, activation='relu', padding='same')(x)
x = Conv1D(32,31, activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Conv1D(10,31, activation='relu', padding='same')(x)
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(10,activation='relu')(x)
x = Dropout(0.4)(x)
ouput = Dense(10, activation='linear')(x)
model = Model(input, ouput)
model.compile(optimizer=Adam(learning_rate=0.003), loss="mean_squared_error", metrics=['accuracy'])
model.fit(X_train, y_train, epochs=2)
print(model.evaluate(X_test, y_test))


X Train shape:  (54092, 10, 1)
X Test shape:  (27089, 10, 1)
Epoch 1/2
 102/1691 [>.............................] - ETA: 10s - loss: nan - accuracy: 0.0000e+00

KeyboardInterrupt: 